# Model Training
Welcome to the 'Model Training and Prediction' notebook, a crucial facet of our project's data science pipeline. In this notebook, we offer a meticulous examination of our rigorous model development process. The pipeline starts by accepting training data, followed by fitting three distinct types of models to it: Random Forest, Gradient Boosted Tree, and XGBoost. The initial stages include encoding categorical variables and executing Recursive Feature Elimination (RFE) for feature selection. This is succeeded by the application of genetic algorithms to hyperparameter tuning, operating in tandem with a cross-validation routine. Subsequently, the best model is selected based on the highest F1 score, indicating the balance between precision and recall. Finally, the selected model is utilized to predict the outcomes for the current week's round of NRL matches. This process is iterative and cyclical, with the potential for revisiting earlier stages based on the model's performance. Let us proceed with this in-depth exploration.

## Set up Environment
This code segment is setting up the environment for the model training pipeline. It begins by importing sys and pathlib - Python libraries used for managing system parameters and file paths, respectively.

The code then updates the system path to include the "functions" directory. This allows for the import of custom modules `modelling_functions`, `model_properties`, and `training_config` which are stored in this directory. These modules contain custom functions and configuration settings that are critical for the later stages of data preprocessing, model training, and prediction.

Following this, the `project_root` variable is defined. This is achieved by using the pathlib library to establish the root directory of the project.

Finally, the `db_path` variable is constructed. This is the relative path to the SQLite database "footy-tipper-db.sqlite", which is located in the "data" directory of the project root. This path will be used for database connectivity throughout the pipeline.

In [1]:
import sys
import pathlib

sys.path.append("functions") 
import modelling_functions as mf
import model_properties as mp
import training_config as tc

# Get to the root directory
project_root = pathlib.Path().absolute().parent.parent

# Now construct the relative path to your SQLite database
db_path = project_root / "data" / "footy-tipper-db.sqlite"

## Get data
Our process starts by establishing the root directory of the project and constructing the relative path to the 'footy-tipper-db.sqlite' database located within the 'data' directory. We then connect to this SQLite database and use a SQL query housed in the 'footy_tipping_data.sql' file, found in the 'sql' directory, to extract the required data. This data is loaded into a pandas DataFrame, footy_tipping_data, serving as the basis for our subsequent modeling activities. Upon successful extraction of the data, we ensure the database connection is closed, maintaining good coding practice and resource management.

In [2]:
training_data = mf.get_training_data(db_path, 'sql/training_data.sql')
training_data

,game_id,round_id,round_name,game_number,game_state_name,start_time,start_time_utc,venue_name,city,crowd,...,away_prev_result_diff,prev_result_diff,home_elo,away_elo,elo_diff,home_elo_prob,away_elo_prob,elo_draw_prob,elo_prob_diff,home_ground_advantage
0,2.020111e+10,1.0,Round 1,1.0,Final,1.584044e+09,1.584004e+09,CommBank Stadium,Sydney,21363.0,...,0.0,0.0,1510.543309,1496.559437,13.983872,0.515806,0.468960,0.015234,0.046846,8.766223
1,2.020111e+10,1.0,Round 1,2.0,Final,1.584122e+09,1.584083e+09,GIO Stadium,Canberra,10610.0,...,0.0,0.0,1515.710707,1466.529375,49.181332,0.553042,0.410595,0.036364,0.142447,9.452456
2,2.020111e+10,1.0,Round 1,3.0,Final,1.584126e+09,1.584090e+09,Queensland Country Bank Stadium,Townsville,22459.0,...,0.0,0.0,1490.406173,1483.396900,7.009273,0.505936,0.478830,0.015234,0.027106,2.220902
3,2.020111e+10,1.0,Round 1,4.0,Final,1.584198e+09,1.584158e+09,McDonald Jones Stadium,Newcastle,10239.0,...,0.0,0.0,1484.839646,1486.546635,-1.706988,0.493587,0.491179,0.015234,0.002408,6.906021
4,2.020111e+10,1.0,Round 1,5.0,Final,1.584207e+09,1.584167e+09,Accor Stadium,Sydney,NaN,...,0.0,0.0,1505.190449,1506.316339,-1.125890,0.494410,0.490356,0.015234,0.004055,3.330532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
722,2.023111e+10,21.0,Round 21,4.0,Final,1.690038e+09,1.690002e+09,Cbus Super Stadium,Gold Coast,15362.0,...,2.0,-4.0,1489.984964,1479.009557,10.975407,0.511551,0.473215,0.015234,0.038335,1.319097
723,2.023111e+10,21.0,Round 21,5.0,Final,1.690047e+09,1.690011e+09,McDonald Jones Stadium,Newcastle,20392.0,...,-14.0,30.0,1514.952402,1528.123609,-13.171207,0.477345,0.507422,0.015234,-0.030077,-6.718729
724,2.023111e+10,21.0,Round 21,6.0,Final,1.690054e+09,1.690018e+09,Queensland Country Bank Stadium,Townsville,20710.0,...,-28.0,102.0,1541.048829,1521.493601,19.555229,0.512441,0.451195,0.036364,0.061246,-0.360000
725,2.023111e+10,21.0,Round 21,7.0,Final,1.690121e+09,1.690085e+09,BlueBet Stadium,Penrith,21525.0,...,-4.0,12.0,1545.682868,1435.666580,110.016288,0.656564,0.343436,0.000000,0.313129,20.278597


## Modelling
During the modelling phase, the `train_and_select_best_model` function, part of our `modelling_functions` module, is invoked. This function initiates the training of three distinct models: XGBoost, Random Forest, and Gradient Boosting Classifier. It takes as input the footy tipping data, predictor variables, the outcome variable, and several configuration settings like whether to use Recursive Feature Elimination (RFE), the number of cross-validation folds, and the optimization metric, all sourced from the `training_config` module.

The function first identifies categorical columns in the feature set for one-hot encoding, creating dummy variables for categorical features. Depending on the choice of using RFE, a feature elimination step may be included in the pipeline. Each model subsequently undergoes hyperparameter tuning using a genetic algorithm, facilitated by the `GASearchCV` function.

All the models are then trained and evaluated through cross-validation. The best model, or `footy_tipper`, is selected based on the superior performance on the chosen optimization metric. Additionally, a `LabelEncoder`(`label_encoder`), used to encode the categorical target variable, is returned. This LabelEncoder is specific to the model that performed best. The selected model, encapsulated in a pipeline with pre-processing steps and hyperparameter tuning, is now ready for the prediction phase.

In [3]:
footy_tipper, label_encoder = mf.train_and_select_best_model(
    training_data, tc.predictors, tc.outcome_var,
    tc.use_rfe, tc.num_folds, tc.opt_metric
)

footy_tipper


Model training: XGBClassifier
gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	100   	0.705719	0.0122402  	0.727662   	0.673954   
1  	131   	0.715056	0.00746598 	0.729051   	0.693226   
2  	131   	0.721279	0.00503812 	0.733179   	0.705621   
3  	132   	0.724404	0.00439329 	0.733179   	0.713888   
4  	147   	0.727364	0.0032794  	0.733179   	0.716665   
5  	130   	0.728991	0.0025022  	0.733179   	0.722154   
6  	136   	0.730687	0.00210312 	0.733198   	0.720822   
7  	147   	0.731735	0.00117852 	0.735947   	0.727662   
8  	150   	0.7324  	0.00139817 	0.735947   	0.727662   
9  	132   	0.733161	0.00133712 	0.735947   	0.73042    
10 	143   	0.734155	0.00142505 	0.737298   	0.73043    
11 	132   	0.735175	0.00149758 	0.737298   	0.72906    
12 	140   	0.735861	0.000904034	0.737298   	0.733198   
13 	144   	0.73612 	0.000787782	0.737298   	0.731809   
14 	137   	0.736528	0.000696377	0.737298   	0.734568   
15 	149   	0.736851	0.000817982	0.737308   	0.731809   
16 	142   	0.7371

Pipeline(steps=[('one_hot_encoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['round_name', 'venue_name',
                                                   'city', 'team_home',
                                                   'team_away',
                                                   'broadcast_channel1',
                                                   'broadcast_channel2',
                                                   'broadcast_channel3',
                                                   'game_day'])])),
                ('feature_elimination',
                 RFECV(cv=5,
                       estimator=XGBClassifier(base_score=Non...
                                        'learning_rate': <sklearn_genetic.space.space.Continuous object at 0x147b4f490>,
                                        'max_depth': <sklearn_genetic.space.space.Integer object at 0x147b4f7d0>,
                                        'n_estimators': <sklearn_genetic.space.space.Integer object at 0x14617e9d0>,
                                        'subsample': <sklearn_genetic.space.space.Continuous object at 0x147b4e310>},
                            population_size=100, return_train_score=True,
                            scoring='accuracy'))])

### Display feature importance
The `get_feature_importance` function retrieves feature importances from a trained scikit-learn pipeline. It accounts for different transformations, such as one-hot encoding and recursive feature elimination. The function then returns a sorted DataFrame listing each feature alongside its respective importance, aiding in understanding the model's decision-making process.

In [4]:
# feature_importance_df = mp.get_feature_importances_from_pipeline(footy_tipper, tc.predictors)
# feature_importance_df

## Save Model
The `save_models` function stores the trained LabelEncoder and Pipeline objects to the disk. This allows for easy retrieval and reuse in future model prediction tasks, without the need to retrain these components. The objects are stored in a designated 'models' directory under the project root path, ensuring organized and consistent storage.

In [5]:
mf.save_models(label_encoder, footy_tipper, project_root)